# Luke McMeans - Special Pop Lab

IMPORTANT NOTES:
- Due to issues with using AviationStack not properly accessing tail numbers, I choose to use AeroAPI.
- Taylor Swift's Plane Infomation isn't accessible through AeroAPI or pyflightdata since she requested to stop public tracking. It's not included in this notebook.
    - Unfortuantley, I couldn't find another celebrity who had publicly available flight data for this API or pyflightdata. I've instead added the tail numbers of planes I found on https://globe.adsbexchange.com/, along with a flight I recently took.

In [33]:
import pandas as pd
import requests

AeroAPI Key:

In [34]:
api_key = ""

In [35]:
# array for tail numbers we want to track
tail_numbers = [
    'N350XX',
    'N62892',
    'N474EP',
    'N2227Z',
    'N968RC',
    'N19117',
    'N8841L',
    'N190UW',
    'N65832'
]

In [41]:
data = []
for tail_number in tail_numbers:
    url = f"https://aeroapi.flightaware.com/aeroapi/flights/{tail_number}"
    headers = {
        'x-apikey': api_key
    }
    response = requests.get(url, headers=headers)
    data.extend([
        {
            "tail_number": flight["registration"],
            "origin": flight["origin"]["name"],
            "destination": flight["destination"]['name'],
            "departure_time": flight["actual_off"],
            "arrival_time": flight["actual_on"],
            "aircraft_type": flight["aircraft_type"],
            "status": flight["status"],
        }
        for flight in response.json()['flights'] if flight["actual_on"] is not None and flight["status"] != "result unknown"
    ])

In [42]:
# turn into a dataframe and show it
df = pd.DataFrame(data)

# create consistency with arrivals 
df['status'] = df['status'].replace('Arrived / Gate Arrival', 'Arrived')
df

,tail_number,origin,destination,departure_time,arrival_time,aircraft_type,status
0,N350XX,Fort Lauderdale Exec,Fort Lauderdale Intl,2025-02-24T14:28:13Z,2025-02-24T14:40:13Z,CL30,Arrived
1,N350XX,Chicago Exec,Fort Lauderdale Exec,2025-02-19T22:17:13Z,2025-02-20T00:56:48Z,CL30,Arrived
2,N350XX,Fort Lauderdale Exec,Chicago Exec,2025-02-19T18:53:03Z,2025-02-19T21:45:34Z,CL30,Arrived
3,N350XX,Witham Fld,Fort Lauderdale Exec,2025-02-17T05:39:46Z,2025-02-17T06:02:59Z,CL30,Arrived
4,N350XX,McClellan-Palomar,Witham Fld,2025-02-17T00:49:00Z,2025-02-17T04:55:29Z,CL30,Arrived
...,...,...,...,...,...,...,...
89,N65832,Austin-Bergstrom Intl,Houston Bush Int'ctl,2025-02-20T01:53:04Z,2025-02-20T02:29:00Z,B739,Arrived
90,N65832,Houston Bush Int'ctl,Austin-Bergstrom Intl,2025-02-19T23:08:00Z,2025-02-19T23:37:56Z,B739,Arrived
91,N65832,Washington Dulles Intl,Houston Bush Int'ctl,2025-02-19T18:21:43Z,2025-02-19T21:36:34Z,B739,Arrived
92,N65832,Orlando Intl,Washington Dulles Intl,2025-02-19T13:27:38Z,2025-02-19T15:02:00Z,B739,Arrived


Specifically for Tom Cruise:

In [43]:
df[df['tail_number'] == 'N350XX']

,tail_number,origin,destination,departure_time,arrival_time,aircraft_type,status
0,N350XX,Fort Lauderdale Exec,Fort Lauderdale Intl,2025-02-24T14:28:13Z,2025-02-24T14:40:13Z,CL30,Arrived
1,N350XX,Chicago Exec,Fort Lauderdale Exec,2025-02-19T22:17:13Z,2025-02-20T00:56:48Z,CL30,Arrived
2,N350XX,Fort Lauderdale Exec,Chicago Exec,2025-02-19T18:53:03Z,2025-02-19T21:45:34Z,CL30,Arrived
3,N350XX,Witham Fld,Fort Lauderdale Exec,2025-02-17T05:39:46Z,2025-02-17T06:02:59Z,CL30,Arrived
4,N350XX,McClellan-Palomar,Witham Fld,2025-02-17T00:49:00Z,2025-02-17T04:55:29Z,CL30,Arrived
5,N350XX,Van Nuys,McClellan-Palomar,2025-02-16T22:41:44Z,2025-02-16T23:05:37Z,CL30,Arrived
6,N350XX,McClellan-Palomar,Van Nuys,2025-02-16T00:47:13Z,2025-02-16T01:15:08Z,CL30,Arrived
7,N350XX,Palm Springs Intl,McClellan-Palomar,2025-02-15T23:31:24Z,2025-02-15T23:54:36Z,CL30,Arrived
8,N350XX,Teterboro,Palm Springs Intl,2025-02-15T17:09:25Z,2025-02-15T22:53:49Z,CL30,Arrived
9,N350XX,Washington Dulles Intl,Teterboro,2025-02-14T20:37:29Z,2025-02-14T21:34:10Z,CL30,Arrived


Finding airports that are common between planes.

In [44]:
# retrieving all airports for each plane
airport_arr = []
for tail_number in tail_numbers:
    tail_df = df[df['tail_number'] == tail_number]
    origins = tail_df['origin'].tolist()
    destinations = tail_df['destination'].tolist()
    unique = set(origins + destinations)
    airport_arr.extend(list(unique))

# find duplicates
duplicates = [airport for airport in airport_arr if airport_arr.count(airport) > 1]

# print duplicates
print(f"Common airports as an origin/destination:")
for airport in set(duplicates):
    print(f"- {airport}")

Common airports as an origin/destination:
- Cancun Int'l
- Dallas-Fort Worth Intl
- Denver Intl
- San Francisco Int'l
- San Diego Intl
- Palm Springs Intl
- Washington Dulles Intl
- Orlando Intl
- Los Angeles Intl
- Houston Bush Int'ctl
